In [136]:
import dkfinance_modeller.aktieskat.skat as skat
from dkfinance_modeller.aktieskat.skat import Skat

Her kommer der en meget simpel model, hvor det er antaget at udbytte bliver beskattet med 27% og realisering bliver beskattet med 42%.
Dette er selvfølgelig ikke hvordan beskatning virker rigtigt.

In [137]:
def simpel_model(
                 start_kapital: float, 
                 åop: float, 
                 årligt_afkast: float, 
                 antal_år: int, 
                 udbytte_procent: float,
                 udbytte_skat: float,
                 realisations_skat: float
                ):
    """
    Simpelmodel for beregning af afkast efter et antal år.
    
    Args:
      start_kapital: Start kapital i depotet.
      åop: ÅOP af depotet.
      skatteklasse: Klasse der beskriver hvordan skatten behandles.
      årligt_afkast: Gennemsnitligt årligt afkast.
      antal_år: Antal år.
      udbytte_procent: Hvor meget af afkastet kommer som udbytte.

    Returns:
      Total depotværdi efter antal år efter skat.
    """
    realiseret = start_kapital
    depot_værdi = start_kapital
    
    for _ in range(antal_år):
        # Beregner udbytte.
        udbytte = udbytte_procent * depot_værdi * (årligt_afkast - åop)
        # Beregner beskatning af udbytte.
        beskatning = udbytte_skat*udbytte
        # Holder regnskab på hvor meget af depotet der allerede er blevet beskattet.
        realiseret += udbytte - beskatning
        # Ny depot værdi efter år N.
        depot_værdi = depot_værdi + depot_værdi*(årligt_afkast - åop)*(1-udbytte_procent) + udbytte - beskatning
    # Beregninger beskatning ved realisering af hele depotet,
    # her bliver det brugt at der er holdt styr på hvor meget der er realiseret.
    beskatning = realisations_skat*(depot_værdi - realiseret)
    return depot_værdi - beskatning


Din model A. 

In [138]:
simpel_model(1, 0.3/100, 0.1, 20, 0.8, 0.27, 0.42)

3.9745052517313533

Din model B.

In [139]:
simpel_model(1, 0.6/100, 0.1, 20, 0.2, 0.27, 0.42)

3.8983342410021287

Som det kan ses vil model A give størst afkast inden for den urealistiske skatteopsætning der er lavet.
Nu konstrueret en simpel gennemsnitsmodel med rigtig behandling af progræssionsgrænsen for beskatning.

In [140]:
def simpel_model_rigtig_beskatning(
                 start_kapital: float, 
                 åop: float, 
                 skatteklasse: Skat, 
                 årligt_afkast: float, 
                 antal_år: int, 
                 udbytte_procent: float
                ):
    """
    Simpelmodel for beregning af afkast efter et antal år.
    
    Args:
      start_kapital: Start kapital i depotet.
      åop: ÅOP af depotet.
      skatteklasse: Klasse der beskriver hvordan skatten behandles.
      årligt_afkast: Gennemsnitligt årligt afkast.
      antal_år: Antal år.
      udbytte_procent: Hvor meget af afkastet kommer som udbytte.

    Returns:
      Total depotværdi efter antal år efter skat.
    """
    realiseret = start_kapital
    depot_værdi = start_kapital
    
    for _ in range(antal_år):
        udbytte = udbytte_procent * depot_værdi * (årligt_afkast - åop)
        beskatning = skatteklasse.beregn_skat(udbytte)
        realiseret += udbytte - beskatning
        depot_værdi = depot_værdi + depot_værdi*(årligt_afkast - åop)*(1-udbytte_procent) + udbytte - beskatning
    beskatning = skatteklasse.beregn_skat(depot_værdi - realiseret)
    return depot_værdi - beskatning

Parametre for model A, bemærk at jeg har valgt 100 000 DKK som start kapital.

In [146]:
beskatning = skat.Skat(beskatningstype="aktie")
simpel_model_rigtig_beskatning(100000, 0.3/100, beskatning, 0.1, 20, 0.8)

405745.52517313533

Parametre for model B, bemærk at jeg har valgt 100 000 DKK som start kapital.

In [147]:
beskatning = skat.Skat(beskatningstype="aktie")
simpel_model_rigtig_beskatning(100000, 0.6/100, beskatning, 0.1, 20, 0.2)

398128.4241002131

For dette specifikke start kapital har model A højest afkast, for andre parametere vil model B kunne have højst afkast. 
Man kan ikke sige at det ene altid er bedre end det andet.

In [148]:
beskatning = skat.Skat(beskatningstype="aktie")
simpel_model_rigtig_beskatning(300000, 0.3/100, beskatning, 0.1, 20, 0.8)

1176150.4644858742